## REGENERATION
## TEAM 3

# Preprocessing/Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from scipy import stats
from difflib import SequenceMatcher

In [2]:
df = pd.read_excel("mpg.data.xlsx")

In [3]:
# used to select correct columns
df.drop(df.iloc[:, 9:13], inplace = True, axis = 1)
df

,mpg,cylinders,displayments,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
402,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
403,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
404,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [4]:
#Κύλινδροι 3->4 και 5-->6
df['cylinders'].replace({3: 4, 5: 6}, inplace=True)

In [5]:
#Μετατροπή origin σε string αφού είναι nominal και όχι ordinal
df['origin'] = df['origin'].astype('str')

## Missing Values

In [6]:
# Χρησιμοποιούμε τον SVR regressor ώστε να συμπληρώσουμε τα missing values του horsepower

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

df1 = df.dropna()

sourceData = df1[['mpg','acceleration','cylinders', 'displayments', 'weight']]
targetData = df1[['horsepower']]

xTrain, xTest, yTrain, yTest = train_test_split(sourceData, targetData)
horsePowerClassifier = SVR(kernel='linear')
horsePowerClassifier = horsePowerClassifier.fit(xTrain, yTrain.values.flatten())
score = horsePowerClassifier.score(xTest, yTest.values.flatten())
print(f'score: {score}')

score: 0.9268797956206636


In [7]:
# Εύρεση των missing horsepower values
for rowId in df.index[df['horsepower'].isna()]:
    row = df.iloc[rowId]
    values = row[['mpg', 'acceleration','cylinders', 'displayments', 'weight']]
    prediction = horsePowerClassifier.predict([values])[0]
    df.at[rowId, 'horsepower'] = prediction

df.isna().sum()

mpg             8
cylinders       0
displayments    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [8]:
#Εφόσον πλέον έχουμε null μόνο στο mpg για τη δημιουργία ενός regression του mpg, οι στήλες με missing values στο mpg
#δεν βοηθούν
df.dropna()

,mpg,cylinders,displayments,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
402,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
403,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
404,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [9]:
df.reset_index(drop=True)

,mpg,cylinders,displayments,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
402,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
403,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
404,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


## One-hot encoding

In [ ]:
#Του origin / getdummies

## Scaling/Standardization

In [ ]:
#οχι τώρα αλλά αργότερα

In [ ]:
#Τελικό αρχείο
df.to_csv("extracted.data.csv")